# 深度学习计算
在本章中，我们将深入探索深度学习计算的关键组件， 即模型构建、参数访问与初始化、设计自定义层和块、将模型读写到磁盘， 以及利用GPU实现显著的加速。 这些知识将使读者从深度学习“基础用户”变为“高级用户”。

## 模型构造

在深度学习中，块是指由一层或多层神经网络组成的组件。它允许我们将复杂的神经网络结构分解为更小的、更易于管理的部分。每个块可以执行特定的功能，如特征提取、降维、分类等。通过将不同的块组合在一起，我们可以创建出具有强大学习和表示能力的深度学习模型。

从编程的角度来看，块由类（class）表示。 它的任何子类都必须定义一个将其输入转换为输出的前向传播函数， 并且必须存储任何必需的参数。 注意，有些块不需要任何参数。 最后，为了计算梯度，块必须具有反向传播函数。 在定义我们自己的块时，由于自动微分 提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

我们先回顾一下多层感知机）的代码。

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

# 包含一个具有256个单元和ReLU激活函数的全连接隐藏层， 然后是一个具有10个隐藏单元且不带激活函数的全连接输出层。
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.2957, -0.1580, -0.1441,  0.1106,  0.1475,  0.0851,  0.0494,  0.0813,
          0.0594, -0.0870],
        [ 0.1876, -0.0722, -0.1385,  0.0900,  0.0947,  0.0330,  0.0284,  0.0260,
         -0.0755, -0.1958]], grad_fn=<AddmmBackward0>)

在这个例子中，我们通过实例化`nn.Sequential`来构建我们的模型，
层的执行顺序是作为参数传递的。
简而言之，(**`nn.Sequential`定义了一种特殊的`Module`**)，
即在PyTorch中表示一个块的类，
它维护了一个由`Module`组成的有序列表。
注意，两个全连接层都是`Linear`类的实例，
`Linear`类本身就是`Module`的子类。
另外，我们通过`net(X)`调用我们的模型来获得模型的输出。
这实际上是`net.__call__(X)`的简写，内部会自动执行前向传播。
这个前向传播函数非常简单：
它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。

###  自定义块
在实现我们自定义块之前，我们简要总结一下每个块必须提供的基本功能。
1. 将输入数据作为其前向传播函数的参数。
1. 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。
1. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
1. 存储和访问前向传播计算所需的参数。
1. 根据需要初始化模型参数。

In [3]:
from torch import nn
from torch.nn import functional as F

# MLP类继承了表示块的类
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
        
    # # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self,X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.0252,  0.0586, -0.1102,  0.0751, -0.0104,  0.0304, -0.2154, -0.0178,
          0.0019,  0.1334],
        [-0.0383,  0.1141, -0.2240, -0.0026,  0.1134, -0.0567, -0.0893,  0.0066,
          0.0319,  0.1294]], grad_fn=<AddmmBackward0>)

### 顺序块

实现一个与`Sequential`功能相似的块。`Sequential`的设计是为了把其他模块串起来。
为了构建我们自己的简化的`MySequential`，
我们只需要定义两个关键函数：

1. 一种将块逐个追加到列表中的函数；
1. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

每个Module都有一个_modules属性，_modules的主要优点是： 在模块的参数初始化过程中， 系统知道在_modules字典中查找需要初始化参数的子块。

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [8]:
# 使用我们自己定义的顺序块 
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0774, -0.3473, -0.1783, -0.0619, -0.0510, -0.1176, -0.2447,  0.0689,
          0.0095,  0.2464],
        [ 0.0991, -0.2409, -0.2293,  0.0501, -0.0160, -0.0635, -0.1281,  0.0065,
          0.0985,  0.1977]], grad_fn=<AddmmBackward0>)

### 在前向传播函数中执行代码

有时我们可能希望合并既不是上一层的结果也不是可更新参数的项， 我们称之为常数参数（constant parameter）。

例如，我们需要一个计算函数
$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$的层，
其中$\mathbf{x}$是输入，
$\mathbf{w}$是参数，
$c$是某个在优化过程中没有更新的指定常量。

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

在这个FixedHiddenMLP模型中，我们实现了一个隐藏层， 其权重（self.rand_weight）在实例化时被随机初始化，之后为常量。 这个权重不是一个模型参数，因此它永远不会被反向传播更新。 然后，神经网络将这个固定层的输出通过一个全连接层。

注意，在返回输出之前，模型做了一些不寻常的事情：
它运行了一个while循环，在$L_1$范数大于$1$的条件下，
将输出向量除以$2$，直到它满足条件为止。
最后，模型返回了`X`中所有项的和。

这里只是演示，不一定非要这么做

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(-0.3223, grad_fn=<SumBackward0>)

### 嵌套块

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.4193, grad_fn=<SumBackward0>)

### 小结

* 一个块可以由许多层组成；一个块可以由许多块组成。
* 块可以包含代码。
* 块负责大量的内部处理，包括参数初始化和反向传播。
* 层和块的顺序连接由`Sequential`块处理。

## 参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。 此时，我们的目标是找到使损失函数最小化的模型参数值。 经过训练后，我们将需要使用这些参数来做出未来的预测。

本节，我们将介绍以下内容：

* 访问参数，用于调试、诊断和可视化；
* 参数初始化；
* 在不同模型组件间共享参数。

我们首先看一下具有单隐藏层的多层感知机。

In [12]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0827],
        [-0.0209]], grad_fn=<AddmmBackward0>)

###  参数访问

我们从已有模型中访问参数。 当通过Sequential类定义模型时， 我们可以通过索引来访问模型的任意层。 这就像模型是一个列表一样，每层的参数都在其属性中。 

In [13]:
# 检查第二个全连接层的参数
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.3533,  0.0904,  0.0796, -0.1262,  0.0314,  0.0165, -0.0221, -0.1721]])), ('bias', tensor([-0.0868]))])


#### 目标参数
访问模型具体某一层的参数，通过字典访问

In [16]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].weight.grad == None)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0868], requires_grad=True)
tensor([-0.0868])
True


#### 一次性访问所有参数

通过 .named_parameters() 方法，可以获取每个参数的名称及其对应的张量形状。它返回一个迭代器，其中每个元素是一个元组，包含参数的名称和参数本身（一个 Tensor 对象）。

In [17]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


也可以通过.state_dict()查看，这个方法返回一个包含模型所有权重和偏置的字典。

In [21]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.3671, -0.4369, -0.0284, -0.4646],
                      [-0.4833,  0.3435,  0.3080,  0.1832],
                      [-0.3523, -0.2193, -0.4342,  0.4728],
                      [-0.1467, -0.3278, -0.3771,  0.1030],
                      [-0.4927, -0.0544, -0.1226, -0.4228],
                      [-0.2052,  0.0317, -0.2381,  0.2881],
                      [-0.2318, -0.2508, -0.4626,  0.3863],
                      [ 0.1460, -0.0357, -0.1638, -0.0190]])),
             ('0.bias',
              tensor([-0.0872,  0.0024, -0.1970,  0.0468, -0.0976,  0.1706,  0.1079, -0.1310])),
             ('2.weight',
              tensor([[-0.3533,  0.0904,  0.0796, -0.1262,  0.0314,  0.0165, -0.0221, -0.1721]])),
             ('2.bias', tensor([-0.0868]))])

#### 从嵌套块收集参数

In [22]:
# 定义一个生成块的函数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0828],
        [0.0828]], grad_fn=<AddmmBackward0>)

In [23]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [25]:
# 通过索引访问： 第一个主要的块中、第二个子块的第一层的偏置项
rgnet[0][1][0].bias.data

tensor([-0.4369, -0.1162,  0.3699,  0.0328,  0.2302,  0.0987,  0.2278,  0.3333])

### 参数初始化

默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵， 这个范围是根据输入和输出维度计算出的。 PyTorch的nn.init模块提供了多种预置初始化方法。

#### 内置初始化
nn.init 它包含了多种用于初始化神经网络权重和偏置的方法。以下是nn.init中常用的一些初始化方法：
- normal：此方法从给定均值和标准差的正态分布中生成值来填充张量。用户可以指定正态分布的均值（mean）和标准差（std），默认值分别为0和1。
- constant：使用此方法，可以用一个常数值（val）来填充整个张量。这对于需要特定值初始化的场景非常有用。
- uniform：该方法从均匀分布U(a, b)中生成值来填充输入的张量。其中，a是分布的下界，b是分布的上界。默认情况下，a为0，b为1。
- xavier_uniform 和 xavier_normal：这两种方法基于Xavier初始化策略，也称为Glorot初始化。它们分别根据均匀分布和正态分布来初始化权重，旨在解决权重初始化时可能出现的梯度消失或爆炸问题。
- kaiming_uniform 和 kaiming_normal：这两种方法基于He初始化策略，也称为Kaiming初始化。与Xavier初始化类似，但更适合于ReLU激活函数及其变体。它们也分别使用均匀分布和正态分布。
- sparse：此方法可以将张量中的一部分元素初始化为零，从而实现稀疏初始化。这有助于减少模型复杂度并可能提高泛化能力。
- zeros：函数用于将张量的所有元素初始化为0。这个函数通常用于偏置（bias）的初始化

In [ ]:
# 将所有权重参数初始化为标准差为0.01的高斯随机变量， 且将偏置参数设置为0。
def init_normal(m):
    if type(m) == nn.Linear:
        
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

In [26]:
# 将所有参数初始化为给定的常数，比如初始化为1。
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [27]:
# 对某些块应用不同的初始化方法。
# 使用Xavier初始化方法初始化第一个神经网络层， 然后将第三个神经网络层初始化为常量值42。
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5759, -0.3191, -0.5171,  0.6523])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


#### 自定义初始化
通过实现了一个my_init函数来应用到net

In [28]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  0.0000,  0.0000,  5.5615],
        [ 0.0000, -6.8724,  0.0000,  7.5669]], grad_fn=<SliceBackward0>)

通过直接设置参数的方式

In [29]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000,  6.5615])

### 参数绑定

有时我们希望在多个层间共享参数： 我们可以定义一个层，然后使用它的参数来设置另一个层的参数。

In [30]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


第三个和第五个神经网络层的参数是绑定的。 它们不仅值相等，而且由相同的张量表示。因此，如果我们改变其中一个参数，另一个参数也会改变。

当参数绑定时，梯度会发生什么情况？ 答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层 （即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。

## 延后初始化
你可能会奇怪，在前面的章节中，
- 我们定义了网络架构，但没有指定输入维度。
- 我们添加层时没有指定前一层的输出维度。
- 我们在初始化参数时，甚至没有足够的信息来确定模型应该包含多少参数。

代码能够运行诀窍是深度学习框架的延后初始化（defers initialization）， 即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。

In [31]:
import torch
from torch import nn
from d2l import torch as d2l

net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

F:\workspace\llms-notebooks\pytorch\env\pytorch\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [32]:
net[0].weight

<UninitializedParameter>

In [33]:
X = torch.rand(2, 20)
net(X)

net[0].weight.shape

torch.Size([256, 20])

一旦我们知道输入维度 20，框架就可以通过插入值 20 来识别第一层权重矩阵的形状。识别出第一层的形状后，框架将进入第二层，依此类推计算图直到所有形状都已知。请注意，在这种情况下，只有第一层需要延迟初始化，但框架会顺序初始化。一旦知道所有参数形状，框架就可以最终初始化参数。

## 自定义层

层是神经网络中最基本的构建单元。每个层负责接收输入数据，对其进行特定类型的变换（如线性变换、非线性变换、卷积、池化等），然后输出变换后的结果。典型的层包括全连接层（Dense Layer）、卷积层（Convolutional Layer）、池化层（Pooling Layer）、激活层（Activation Layer）等。

功能：单个层通常执行单一类型的计算操作，比如卷积层用于提取特征，激活层引入非线性，全连接层用于整合特征等。

有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。 在这些情况下，必须构建自定义层。本节将展示如何构建自定义层。


### 不带参数的层

In [36]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    # 从其输入中减去均值
    def forward(self, X):
        return X - X.mean()

In [37]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [38]:
# 将层作为组件合并到更复杂的模型中
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [39]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(9.3132e-10, grad_fn=<MeanBackward0>)

### 带参数的层

下面我们定义具有参数的层， 这些参数可以通过训练进行调整。 我们可以使用**内置函数**来创建参数，这些函数提供一些基本的管理功能。 比如管理访问、初始化、共享、保存和加载模型参数。

In [40]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [42]:
# 实例化MyLinear类并访问其模型参数
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.8507, -0.5058,  0.3367],
        [-1.6526,  0.2127, -1.5380],
        [-1.6145, -0.1324, -0.3577],
        [-0.7560,  0.0280,  0.6306],
        [-1.6107, -1.5456,  0.7213]], requires_grad=True)

In [43]:
# 使用自定义层直接执行前向传播计算
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 0.3856],
        [0.0000, 0.0000, 0.0000]])

- 层是神经网络的基本组成单位，执行单一的转换操作。
- 块是由一个或多个层构成的模块，旨在实现更复杂的变换或促进模型结构的复用性。块提供了更高层次的抽象，使得模型设计更加模块化和高效。

## 读写文件

有时我们希望保存训练的模型， 以备将来在各种环境中使用（比如在部署中进行预测）。 还有，当运行一个耗时较长的训练过程时， 最佳的做法是定期保存中间结果， 以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。

### 加载和保存张量

我们可以调用load和save函数分别读写它们。

In [45]:
import torch
from torch import nn
from torch.nn import functional as F

# 存储、加载单个张量
x = torch.arange(4)
torch.save(x, 'data/x-file')

In [46]:
x2 = torch.load('data/x-file')
x2

tensor([0, 1, 2, 3])

In [48]:
# 存储、加载 张量列表 
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [49]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'data/mydict')
mydict2 = torch.load('data/mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 加载和保存模型参数

度学习框架提供了内置函数来保存和加载整个网络。 需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。 

In [50]:
# 多层感知机模型 
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [53]:
# 将模型的参数存储在一个叫做“mlp.params”的文件中
torch.save(net.state_dict(), 'data/mlp.params')

In [52]:
# 恢复模型

# 实例化了原始多层感知机模型
clone = MLP()
#  这里我们不需要随机初始化模型参数，而是直接读取文件中存储的参数。
clone.load_state_dict(torch.load('data/mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [54]:
# 结果验证
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## GPU

使用nvidia-smi命令来查看显卡信息.

In [55]:
!nvidia-smi

Mon Jun 17 14:27:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce 940MX         WDDM  |   00000000:02:00.0 Off |                  N/A |
| N/A    0C    P0             N/A /  200W |       0MiB /   2048MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

在PyTorch中，每个数组都有一个设备（device）， 我们通常将其称为环境（context）。 
默认情况下，所有变量和相关的计算都分配给CPU。 有时环境可能是GPU。

### 计算设备
在PyTorch中，CPU和GPU可以用torch.device('cpu') 和torch.device('cuda')表示。
如果有多个GPU，我们使用`torch.device(f'cuda:{i}')`
来表示第$i$块GPU（$i$从0开始）。
另外，`cuda:0`和`cuda`是等价的。
可以通过`torch.cuda.device_count()`查看GPU数量

In [56]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [57]:
torch.cuda.device_count()

1

In [58]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

### 张量与GPU
我们可以查询张量所在的设备。 默认情况下，张量是在CPU上创建的。

**需要注意的是，无论何时我们要对多个项进行操作， 它们都必须在同一个设备上。**例如，如果我们对两个张量求和， 我们需要确保两个张量都位于同一个设备上， 否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

In [59]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

#### 存储在GPU上

有几种方法可以在GPU上存储张量:
- 使用PyTorch的.to()方法：例如，如果要将一个张量移动到GPU上，可以使用tensor.to('cuda')。
- 在创建张量时直接指定设备：例如，使用torch.tensor(data, device='cuda')可以直接在GPU上创建一个张量。
- 使用torch.cuda模块：

我们可以使用nvidia-smi命令查看显存使用情况。 一般来说，我们需要确保不创建超过GPU显存限制的数据。

In [61]:
# 在创建张量时指定存储设备
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [63]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.9941, 0.8429, 0.5843],
        [0.8334, 0.5671, 0.1547]], device='cuda:0')

#### 复制
如果我们[**要计算`X + Y`，我们需要决定在哪里执行这个操作**]，因为`X 、Y`不在同一设备上，直接相加，运行时引擎会因为在同一设备上找不到数据会导致失败。我们可以通过`.cuda()`复制数据到指定设备上。
如果复制的数据已经在指定的设备上了，再次执行`.cuda()`，并不会重新复制、分配新内存。

In [65]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [66]:
Y + Z

tensor([[1.9941, 1.8429, 1.5843],
        [1.8334, 1.5671, 1.1547]], device='cuda:0')

### 神经网络与GPU

神经网络模型也可以指定设备。 

In [67]:
# 将模型参数放在GPU上
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [68]:
net(X)

tensor([[-0.3958],
        [-0.3958]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [70]:
# 查看模型参数存储位置
net[0].weight.data.device

device(type='cuda', index=0)

### 小结

* 我们可以指定用于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使用CPU进行计算。
* 深度学习框架要求计算的所有输入数据都在同一设备上，无论是CPU还是GPU。
* 不经意地移动数据可能会显著降低性能。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy `ndarray`中）时，将触发全局解释器锁，从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。